In [22]:
import torch
import torch.nn as nn
import torchvision.models as models
import os
import xml.etree.ElementTree as ET
import cv2
from PIL import Image

In [23]:

# downloading the model with weights pretrained
model = models.resnet18(pretrained=True)

#converting the model to a list of modules
model_list = list(model.modules())

#removing the last two average pooling and the fully connected layer
resnetModel = nn.Sequential(*model_list[:-2])

c:\Users\kaasa\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\kaasa\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
# DECLARING ALL PATHS AND VARIABLES.

DATA_DIR = "\\".join(os.getcwd().split("\\")[:-1]) + "\\" + "DataSet"
ANNOTATIONS_DIR = DATA_DIR + "\\" + "Annotations\\"
IMG_DIR = DATA_DIR + "\\" + "Images\\"
PROCESSESED_PATH = DATA_DIR + '\\' + 'ProcessedDatasets\\'
AVAILABLE_CLASSES = ['n02089078-black-and-tan_coonhound','n02091831-Saluki'
                     ,'n02092002-Scottish_deerhound',
                     'n02095314-wire-haired_fox_terrier']


CLASS_NAMES = []
CLASS_CODES = []
ANNOTATION_PATHS = []
IMAGE_PATHS = []
PROCESSED_IMAGE_PATHS = []


for i in range(4):
    CLASS_NAMES.append("-".join(AVAILABLE_CLASSES[i].split("-")[1:]))
    CLASS_CODES.append(AVAILABLE_CLASSES[i].split("-")[0])
    ANNOTATION_PATHS.append(ANNOTATIONS_DIR + AVAILABLE_CLASSES[i] + "\\" )
    IMAGE_PATHS.append(IMG_DIR + AVAILABLE_CLASSES[i] + "\\")
    PROCESSED_IMAGE_PATHS.append(PROCESSESED_PATH + CLASS_CODES[i]+"-" +CLASS_NAMES[i]+ "\\")



In [30]:
# FUNCTION TO PROCESS THE IMAGE BASED ON THE CORRESPONDING ANNOTATIONS.

def get_bounding_boxes(annot):
  xml = annot
  tree = ET.parse(xml)
  root = tree.getroot()
  objects = root.findall('object')
  bbox = []
  for o in objects:
    bndbox = o.find('bndbox')
    xmin = int(bndbox.find('xmin').text)
    ymin = int(bndbox.find('ymin').text)
    xmax = int(bndbox.find('xmax').text)
    ymax = int(bndbox.find('ymax').text)
    bbox.append((xmin,ymin,xmax,ymax))
  return bbox

#FUNCTION TO CROP EVERY IMAGE IN EVERY CLASS AND SAVE IN A PROCESSED DIRECTORY.
def normalize(arr):
    """
    Linear normalization
    http://en.wikipedia.org/wiki/Normalization_%28image_processing%29
    """
    arr = arr.astype('float')
    # Do not touch the alpha channel
    for i in range(3):
        minval = arr[...,i].min()
        maxval = arr[...,i].max()
        if minval != maxval:
            arr[...,i] -= minval
            arr[...,i] *= (255.0/(maxval-minval))
    return arr

def crop_image(image_path , annotation_path,save_path):
  img = cv2.imread(image_path,cv2.IMREAD_COLOR)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  bb = get_bounding_boxes(annotation_path)
  bbox = bb[0]
  cropped_data = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]    # cropping the image
  cropped_data = cv2.resize(cropped_data,dsize=(224 ,224)) # rescaling it to a square image
  #print(normalize(cropped_data))
  
  crop_img = Image.fromarray(normalize(cropped_data).astype('uint8'),'RGB')
  #crop_img = Image.fromarray(cropped_data,'RGB')  # converting the numpy array to an image
  crop_img.save(save_path)
  


  

In [32]:

for i in range(4):
    for dog in os.listdir(IMAGE_PATHS[i]):
        image_path = dog
        annotation_path = dog.split(".")[0]
        if not os.path.exists(PROCESSED_IMAGE_PATHS[i]):
            os.mkdir(PROCESSED_IMAGE_PATHS[i])
        crop_image(IMAGE_PATHS[i]+  image_path, ANNOTATION_PATHS[i] 
                   + annotation_path, PROCESSED_IMAGE_PATHS[i] + dog)



[[[ 53.  20.  13.]
  [ 54.  19.  13.]
  [ 55.  18.  12.]
  ...
  [215. 215. 211.]
  [209. 209. 207.]
  [201. 201. 201.]]

 [[ 56.  22.  14.]
  [ 58.  21.  14.]
  [ 59.  20.  14.]
  ...
  [209. 211. 208.]
  [206. 207. 205.]
  [199. 199. 199.]]

 [[ 57.  20.  12.]
  [ 57.  20.  12.]
  [ 57.  19.  11.]
  ...
  [207. 209. 206.]
  [205. 208. 204.]
  [196. 197. 197.]]

 ...

 [[171.  93.  27.]
  [169.  91.  25.]
  [167.  89.  23.]
  ...
  [152. 129.  87.]
  [141. 120.  77.]
  [139. 118.  75.]]

 [[173.  95.  29.]
  [172.  94.  28.]
  [169.  91.  25.]
  ...
  [122.  99.  58.]
  [126. 106.  62.]
  [132. 111.  68.]]

 [[175.  98.  31.]
  [175.  98.  31.]
  [172.  94.  28.]
  ...
  [135. 111.  72.]
  [126. 104.  62.]
  [107.  84.  43.]]]
[[[244.88095238 245.55555556 196.        ]
  [246.9047619  246.60493827 199.        ]
  [250.95238095 247.65432099 203.        ]
  ...
  [248.92857143 225.61728395 175.        ]
  [246.9047619  222.4691358  176.        ]
  [245.89285714 223.51851852 176.        